In [2]:
import os
import shutil
from IPython import get_ipython

from PIL import Image, ImageOps
import imageio

In [3]:
notebook_path = get_ipython().starting_dir
output_folder = os.path.join(notebook_path, "diagrams")

shutil.rmtree(output_folder, ignore_errors=True)
os.makedirs(output_folder)

In [4]:
big_name = "Azure Serverless Architecture"

In [6]:
from diagrams import Diagram
from diagrams.azure.compute import FunctionApps
from diagrams.azure.integration import EventGrid
from diagrams.azure.database import CosmosDb
from diagrams.azure.storage import BlobStorage
from diagrams.azure.integration import APIManagement
from diagrams.azure.devops import Devops
from diagrams.azure.network import CDN

with Diagram(big_name, filename=os.path.join(output_folder, big_name), show=False, direction="TB"):
    # Networking and CDN
    route53 = Route53("Route 53")
    cloudfront = CloudFront("CloudFront")
    vpc = VPC("VPC")

    # Core Serverless Services
    api_gateway = APIGateway("API Gateway")
    lambda_function = Lambda("AWS Lambda")
    s3 = S3("S3 Bucket")
    dynamodb = Dynamodb("DynamoDB")
    sns = SNS("SNS Topic")
    sqs = SQS("SQS Queue")
    cognito = Cognito("Cognito")

    # Identity and Access Management
    iam = IAM("IAM")

    # Monitoring and Logging
    cloudwatch = Cloudwatch("CloudWatch")

    # Architecture flow
    route53 >> cloudfront >> api_gateway
    api_gateway >> lambda_function >> vpc
    lambda_function >> [s3, dynamodb, sns]
    sns >> sqs
    api_gateway >> cognito
    iam >> lambda_function
    lambda_function >> cloudwatch

In [7]:
def find_largest_dimensions(image_folder, file_names):
    """ Find the largest width and height among all images """
    max_width, max_height = 0, 0
    for file_name in file_names:
        with Image.open(os.path.join(image_folder, file_name)) as img:
            if img.width > max_width:
                max_width = img.width
            if img.height > max_height:
                max_height = img.height
    return max_width, max_height


def pad_image_to_size(img, size):
    """ Pad image with white background to match the given size """
    padded_img = ImageOps.pad(img, size, color="white")
    return padded_img


def create_fading_gif(image_folder, output_gif, duration=5, fade_duration=2, static_duration=200, loop=0):
    images = []
    filenames = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")])

    # Find the largest image size
    max_width, max_height = find_largest_dimensions(image_folder, filenames)
    largest_size = (max_width, max_height)

    for i in range(len(filenames) - 1):
        img1 = Image.open(os.path.join(image_folder, filenames[i]))
        img2 = Image.open(os.path.join(image_folder, filenames[i + 1]))

        # Pad images to the same size
        img1 = pad_image_to_size(img1, largest_size)
        img2 = pad_image_to_size(img2, largest_size)

        # Display static image for a duration
        for _ in range(static_duration):
            images.append(img1)

        # Create fading effect
        for t in range(fade_duration):
            alpha = t / fade_duration
            blended = Image.blend(img1, img2, alpha)
            images.append(blended)

    # Add and display the last image statically
    last_img = Image.open(os.path.join(image_folder, filenames[-1]))
    last_img = pad_image_to_size(last_img, largest_size)
    for _ in range(static_duration):
        images.append(last_img)

    # Save to GIF with infinite loop
    imageio.mimsave(output_gif, images, duration=duration, loop=loop)


image_folder = output_folder
output_gif = f'{big_name}.gif'  # Output GIF file name
create_fading_gif(image_folder, os.path.join(image_folder, output_gif))

IndexError: list index out of range